# Agents: interacting with tools

**Overview:** Connect an LLM to tools, give natural language instructions, and observe how the agent selects, executes, and validates multi-step tasks such as searching, sending, and deleting emails.

**Steps:** 
1) **Define tools** — tools are python functions with clear docstrings.  
2) **Prepare prompt** — the prompt should help the agent perceive the context and available tools .  
3) **Query** — give the agent clear instruction, the agent will come up with a plan and also call the tools itself.  

**Example: email assistant agent**

e.g., Check for unread emails from alice@work.com, mark them as read, and send a polite follow-up. Respond with a summary of the email content and steps you have done.

<img src="lab_overview.png" alt="Example of a calendar assistant" width="700"/>


## 0. Initialization

In [2]:
# ================================
# Imports
# ================================

# --- Third-party ---
from dotenv import load_dotenv
# import aisuite as ai
import json
import pprint

# --- Local / project ---
import utils
import display_functions
import email_tools

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="pydantic")

# ================================
# Environment & Client
# ================================
load_dotenv()          # Load environment variables from .env


True

In [3]:
from google import genai
from google.genai import types
import os

# Configure the client and tools
# gemini_api_key = os.getenv("GEMINI_API_KEY")
gemini_api_key = os.getenv("GEMINI_API_KEY_2")

client = genai.Client(api_key=gemini_api_key)

In [ ]:
def get_response(client, model_name, config, prompt):
    response = client.models.generate_content(
        model=model_name,
        contents=prompt,
        config=config,
    )
    return response

## 1. Simulated email service

### 1.1 Components
This lab uses a **simulated email backend** to mimic real-world email interactions.
Think of it as your personal sandbox email inbox: it comes preloaded with messages so you can practice without sending real emails.  


| Layer                   | Purpose                        |
|-------------------------|--------------------------------|
| **FastAPI**             | Exposes REST endpoints         |
| **SQLite + SQLAlchemy** | Stores and queries emails locally |
| **Pydantic**            | Ensures inputs and outputs are valid |
| **AISuite tools**       | Bridge between the LLM and the service |


### 1.2 Endpoints
The service provides several routes that simulate common email actions.  

- `POST /send` → send a new email  
- `GET /emails` → list all emails  
- `GET /emails/unread` → show only unread emails  
- `GET /emails/{id}` → fetch a specific email by ID  
- `GET /emails/search?q=...` → search emails by keyword  
- `GET /emails/filter` → filter by recipient or date range  
- `PATCH /emails/{id}/read` → mark an email as read  
- `PATCH /emails/{id}/unread` → mark an email as unread  
- `DELETE /emails/{id}` → delete an email by ID  
- `GET /reset_database` → reset emails to initial state (for testing)  

### 1.3 Tools
`Available tools`

| Tool Function                      | Action                                                                 |
|------------------------------------|------------------------------------------------------------------------|
| `list_all_emails()`                | Fetch all emails, newest first                                         |
| `list_unread_emails()`             | Retrieve only unread emails                                            |
| `search_emails(query)`             | Search by keyword in subject, body, or sender                          |
| `filter_emails(...)`               | Filter by recipient and/or date range                                  |
| `get_email(email_id)`              | Fetch a specific email by ID                                           |
| `mark_email_as_read(id)`           | Mark an email as read                                                  |
| `mark_email_as_unread(id)`         | Mark an email as unread                                                |
| `send_email(...)`                  | Send a new (simulated) email                                                |
| `delete_email(id)`                 | Delete an email by ID                                                  |
| `search_unread_from_sender(addr)`  | Return unread emails from a given sender (e.g., `boss@email.com`)      |

For example: all_emails = email_tools.list_all_emails()

`To run email service:` python -m uvicorn email_server.email_service:app --reload --port 8000

In [7]:
# Test sending a new email and fetch it by ID
new_email = email_tools.send_email("test@example.com", "Lunch plans", "Shall we meet at noon?")
content_ = email_tools.get_email(new_email['id'])

# Uncomment the ones you want to try:
#content_ = email_tools.list_all_emails()
#content_ = email_tools.list_unread_emails()
#content_ = email_tools.search_emails("lunch")
#content_ = email_tools.filter_emails(recipient="test@example.com")
#content_ = email_tools.mark_email_as_read(new_email['id'])
#content_ = email_tools.mark_email_as_unread(new_email['id'])
#content_ = email_tools.search_unread_from_sender("test@example.com")
#content_ = email_tools.delete_email(new_email['id'])

utils.print_html(content=json.dumps(content_, indent=2), title="Testing the email_tools")

~ Sending email
	+ Recipient: test@example.com
	+ Subject: Lunch plans
	+ Body: Shall we meet at noon?
~ Retrieving email with ID: 7


### 1.4 Resetting the email service

In [8]:
utils.reset_database()

{'message': 'Database reset and emails reloaded'}

## End-to-end pipeline
### Overview
**Scenario**
> “Check for unread emails from `boss@email.com`, mark them as read, and send a polite follow-up.”

**What happen**
1. The agent interprets your instruction.  
2. It selects the right tools (`search_unread_from_sender` → `mark_email_as_read` → `send_email`).  
3. It executes each action automatically, without asking for confirmation.

**What to look for:**
- A clear **tool-call trace** showing which tools were used and the arguments passed.
- A concise **final message** summarizing the actions performed (e.g., “Found 1 unread email, marked as read, sent follow-up”).

### List of tools

In [9]:
functions = [ 
    email_tools.search_unread_from_sender,
    email_tools.list_unread_emails,
    email_tools.search_emails,
    email_tools.get_email,
    email_tools.mark_email_as_read,
    email_tools.send_email
]

### Prompt

In [10]:
def build_prompt(request_: str) -> str:
    return f"""
- You are an AI assistant specialized in managing emails.
- You can perform various actions such as listing, searching, filtering, and manipulating emails.
- Use the provided tools to interact with the email system.
- Never ask the user for confirmation before performing an action.
- If needed, my email address is "you@email.com" so you can use it to send emails or perform actions related to my account.

{request_.strip()}
"""

In [ ]:
def run_workflow(user_instruction, functions, client, model_name="gemini-2.5-flash-lite"):
    # Configure the client and model
    config = types.GenerateContentConfig(
        tools=functions
    )
    prompt = build_prompt(user_instruction)

    # Make the request
    response = get_response(client, model_name, config, prompt)

    # Print the final, user-facing response
    print("\nAgent response:", response.text)

In [12]:
# Check status before running
email_tools.search_unread_from_sender("alice@work.com")

~ Searching unread emails from sender: alice@work.com


[{'id': 5,
  'sender': 'alice@work.com',
  'recipient': 'you@email.com',
  'subject': 'Lunch?',
  'body': 'Free for lunch today?',
  'timestamp': '2025-12-25T07:08:08.947750',
  'read': False}]

In [13]:
user_instruction = "Check for unread emails from alice@work.com, mark them as read, and send a polite follow-up. Respond with a summary of the email content and steps you have done."
run_workflow(
    user_instruction=user_instruction, 
    functions=functions, 
    client=client,
    model_name="gemini-2.5-flash-lite"
)

~ Searching unread emails from sender: alice@work.com
~ Marking email as read with ID: 5
~ Sending email
	+ Recipient: alice@work.com
	+ Subject: Re: Lunch?
	+ Body: Hi Alice, I am afraid I am busy for lunch today. Can we do it another time? Best, You

Agent response: I found one unread email from alice@work.com with the subject "Lunch?". The email content is "Free for lunch today?". I have marked this email as read and sent a follow-up email to alice@work.com with the subject "Re: Lunch?". The follow-up email body is "Hi Alice, I am afraid I am busy for lunch today. Can we do it another time? Best, You".


In [14]:
# Check status after running
email_tools.search_unread_from_sender("alice@work.com")

~ Searching unread emails from sender: alice@work.com


[]